In [1]:
import pandas as pd
import numpy as np
import requests
data = pd.read_csv("data/output.csv", encoding='utf-8-sig')
data.head()

,ilaçAdı,etkinMaddeAdı,firmaAdı,kübOnayTarihi,ktOnayTarihi,kübDocLink,ktDocLink
0,% 0.4 LİDODEKS % 5 DEKSTROZ İÇİNDE İ.V. İNFÜZY...,LİDOKAİN HİDROKLORÜR,POLİFARMA İLAÇ SAN. VE TİC. A.Ş.,05/08/2016,05/08/2016,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
1,% 0.8 LİDODEKS % 5 DEKSTROZ İÇİNDE İ.V. İNFÜZY...,LİDOKAİN HİDROKLORÜR,POLİFARMA İLAÇ SAN. VE TİCARET A.Ş.,05/08/2016,05/08/2016,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
2,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
3,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
4,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...


In [2]:
data["kübOnayTarihi"] = pd.to_datetime(data["kübOnayTarihi"], format="%d/%m/%Y", errors="coerce")
data["ktOnayTarihi"] = pd.to_datetime(data["ktOnayTarihi"], format="%d/%m/%Y", errors="coerce")


In [3]:
nullLink = "https://titck.gov.tr/storage/kubKtAttachments/"

In [4]:
data["kübDocLink"] = data["kübDocLink"].replace(nullLink, np.nan)
data["ktDocLink"] = data["ktDocLink"].replace(nullLink, np.nan)

In [5]:
data.dtypes

ilaçAdı                  object
etkinMaddeAdı            object
firmaAdı                 object
kübOnayTarihi    datetime64[ns]
ktOnayTarihi     datetime64[ns]
kübDocLink               object
ktDocLink                object
dtype: object

In [6]:
data.iloc[0]

ilaçAdı          % 0.4 LİDODEKS % 5 DEKSTROZ İÇİNDE İ.V. İNFÜZY...
etkinMaddeAdı                                 LİDOKAİN HİDROKLORÜR
firmaAdı                          POLİFARMA İLAÇ SAN. VE TİC. A.Ş.
kübOnayTarihi                                  2016-08-05 00:00:00
ktOnayTarihi                                   2016-08-05 00:00:00
kübDocLink       https://titck.gov.tr/storage/kubKtAttachments/...
ktDocLink        https://titck.gov.tr/storage/kubKtAttachments/...
Name: 0, dtype: object

In [7]:
print(data.duplicated().sum())
data = data.drop_duplicates()
print(data.duplicated().sum())

49
0


In [8]:
sorted_data = data.sort_values(by=["kübOnayTarihi", "ktOnayTarihi"], ascending=False)
latest_data = sorted_data.drop_duplicates(subset=["ilaçAdı", "etkinMaddeAdı", "firmaAdı"])
len(latest_data), latest_data["kübDocLink"].isna().sum(), latest_data["ktDocLink"].isna().sum()

(14618, 75, 74)

In [9]:
for idx, row in latest_data.iterrows():
    original_rows = sorted_data[
        (sorted_data["ilaçAdı"] == row["ilaçAdı"]) &
        (sorted_data["etkinMaddeAdı"] == row["etkinMaddeAdı"]) &
        (sorted_data["firmaAdı"] == row["firmaAdı"])
    ]
    if pd.isna(row["kübDocLink"]):
        # Eğer dropna() boşsa NaN bırak
        küb_links = original_rows["kübDocLink"].dropna()
        if not küb_links.empty:
            latest_data.loc[idx, "kübDocLink"] = küb_links.iloc[0]
    if pd.isna(row["ktDocLink"]):
        # Eğer dropna() boşsa NaN bırak
        kt_links = original_rows["ktDocLink"].dropna()
        if not kt_links.empty:
            latest_data.loc[idx, "ktDocLink"] = kt_links.iloc[0]

len(latest_data), latest_data["kübDocLink"].isna().sum(), latest_data["ktDocLink"].isna().sum()

(14618, 72, 67)

In [10]:
len(latest_data), latest_data["kübDocLink"].isna().sum(), latest_data["ktDocLink"].isna().sum()

(14618, 72, 67)

In [11]:
print(print(latest_data.duplicated().sum()))
latest_data = latest_data.dropna(subset=["kübDocLink", "ktDocLink"], how='all')
len(latest_data)

0
None


14617

In [ ]:
latest_data.to_csv("data/cleanOutput.csv", index=False, encoding='utf-8-sig')